# **CoinMarketCap Dataset For Time Series Analysis**

Cryptocurrencies are a fast adapting medium of digital currency over the globe. These digital currencies are easily available to purchase with authenticity on many different websites, making it accessible to everyone, and with retailers accepting and trading various cryptocurrencies, money market scenarios are changing and going through a major change. Cryptocurrency is based on blockchain technology, which is a revolutionarily distributed digital backbone. Blockchain implements secure, decentralized systems that can aid in conquering organizational issues of trust, privacy, and security that have been around the work culture throughout the ages. 

#Analysis

For these continuous changes of values over a while, time series analysis and forecasting are used. Recurrent Neural Networks and Long Short Term Memory are an important part of machine learning algorithms being used for time-series predictions.

For demonstration, we will be using complete cryptocurrency market history data from [Kaggle](https://www.kaggle.com/taniaj/cryptocurrency-market-history-coinmarketcap), which has data scrapped from CoinMarketCap 2014 to 2018 containing 887 cryptos token information. Now let’s do some time series analysis on this data to infer insights out of it. 

In [ ]:

!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras torch torchvision \
    tqdm scikit-image --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


In [ ]:
# !gdown https://drive.google.com/uc?id=1eAiGJyJJWaMWuEQJlnLYmBNu1x9CmZI0

In [ ]:
# !unzip all_currencies.csv.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("https://gitlab.com/AnalyticsIndiaMagazine/practicedatasets/-/raw/main/time_series_coin_market_data/all_currencies.csv",parse_dates=["Date"], index_col="Date")
df.head(5)

In [ ]:
df.index

The dataset consists of 8 attributes:


> * Date: The date of observation.
> * Symbol: The symbol of a particular crypto token.
> * Open: The opening price of the stock of each day.
> * High: The highest value over the period.
> * Low: The lowest value over the period.
> * Close: The closing price each day of the stock.
> * Volume: The total amount of security changes over a given period. 
> * Market Cap: It is the total amount in the dollar market value of a company’s outstanding shares.

In [ ]:
df.describe()

In [ ]:
df.shape, df.info()

In [ ]:
ax = df.groupby(['Symbol'])['Market Cap'].last().sort_values(ascending=False).head(10).sort_values().plot(kind='barh');
ax.set_xlabel("Market cap (in billion USD)");
plt.title("Top 10 Currencies by Market Cap");

In [ ]:
ax = df.groupby(['Symbol'])['Volume'].last().sort_values(ascending=False).head(10).sort_values().plot(kind='barh');
ax.set_xlabel("Transaction Volume (in million)");
plt.title("Top 10 Currencies by Transaction Volume");

In [ ]:
top_5_currency_names = df.groupby(['Symbol'])['Market Cap'].last().sort_values(ascending=False).head(5).index
data_top_5_currencies = df[df['Symbol'].isin(top_5_currency_names)]
data_top_5_currencies.head(5)

In [ ]:
ax = data_top_5_currencies.groupby(['Date', 'Symbol'])['Close'].mean().unstack().plot();
ax.set_ylabel("Price per 1 unit (in USD)");
plt.title("Price per unit of currency");

In [ ]:
ax = data_top_5_currencies.groupby(['Date', 'Symbol'])['Market Cap'].mean().unstack().plot();
ax.set_ylabel("Market Cap (in billion USD)");
plt.title("Market cap per Currency");

In [ ]:
ax = data_top_5_currencies.groupby(['Date', 'Symbol'])['Volume'].mean().unstack().plot();
ax.set_ylabel("Transaction Volume (in million)");
plt.title("Transaction Volume per Currency");

In [ ]:
%matplotlib inline
df['Close'].plot()

In [ ]:
df['Close'].resample('Y').mean().plot(kind='bar')

In [ ]:
from pandas.plotting import lag_plot

lag_plot(df['Volume'].tail(250))

In [ ]:
df['Volume'].resample('Y').mean().plot.bar()

In [ ]:
df_BTC = df.loc[df['Symbol'] == 'BTC',['Close']]
df_BTC.shape

In [ ]:
train = df_BTC[:'2018'].values
valid = df_BTC['2018':].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_BTC)

In [ ]:
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=2, batch_size=1, verbose=1)

In [ ]:
inputs = df_BTC[len(df_BTC) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

In [ ]:
X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
train = df_BTC[:'2018']
valid = df_BTC['2018':]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])

#**Related Articles:**

> * [CoinMarketCap Dataset for Time Series Analysis](https://analyticsindiamag.com/guide-to-coinmarketcap-dataset-for-time-series-analysis-historical-prices-of-all-cryptocurrencies/)

> * [Time Series Data Analysis on Forex Historical Dataset](https://analyticsindiamag.com/applying-time-series-analysis-on-forex-historical-dataset/)

> * [Analyzing Climate Change](https://analyticsindiamag.com/time-series-analysis-on-climate-change/)

> * [TadGAN](https://analyticsindiamag.com/hands-on-guide-to-tadgan-with-python-codes/)

> * [Pastas](https://analyticsindiamag.com/guide-to-pastas-a-python-framework-for-hydrogeological-time-series-analysis/)

> * [Bitcoin Price Prediction](https://analyticsindiamag.com/guide-to-implementing-time-series-analysis-predicting-bitcoin-price-with-rnn/)